In [ ]:
!git clone https://github.com/SkalskiP/yolov9.git
%cd yolov9
!pip install -r requirements.txt -q
!pip install telebot


Cloning into 'yolov9'...
remote: Enumerating objects: 325, done.
remote: Counting objects: 100% (325/325), done.
remote: Compressing objects: 100% (152/152), done.
remote: Total 325 (delta 165), reused 264 (delta 162), pack-reused 0
Receiving objects: 100% (325/325), 2.23 MiB | 7.06 MiB/s, done.
Resolving deltas: 100% (165/165), done.
/content/yolov9
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.6/240.6 kB 5.4 MB/s eta 0:00:00


In [ ]:
import telebot
import shutil


import os
import telebot
from telebot import types
BOT_TOKEN = '7094092093:AAHrwOOL2BWiORsteQ6pMV3bVzmAEK5Vf1A'
bot = telebot.TeleBot(BOT_TOKEN)
output_folder="/content/runs/detect/predict"
@bot.message_handler(content_types=['photo'])
def echo_photo(message):
    print("Image Received")
    # Get the photo ID of the largest available photo
    photo = message.photo[-1]
    # Download the photo
    photo_info = bot.get_file(photo.file_id)
    photo_url = f"https://api.telegram.org/file/bot{BOT_TOKEN}/{photo_info.file_path}"

    # Define the folder where images will be saved
    save_folder = "/content/yolov9/saved_images"
    try:
        shutil.rmtree("/content/yolov9/saved_images")
        shutil.rmtree("/content/yolov9/runs/detect/exp")
        shutil.rmtree("/content/yolov9/runs/detect/exp2")
        shutil.rmtree("/content/yolov9/runs/detect/exp3")
        #print(f"Directory '{directory_path}' and its contents have been removed.")
    except Exception as e:
        print(f"Error while removing directory: {e}")

    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    # Define the image filename
    image_filename = f"{message.from_user.id}_{message.message_id}.jpg"
    image_path = os.path.join(save_folder, image_filename)

    os.system(f"wget -O {image_path} {photo_url}")
    !python detect.py \
    --img 1280 --conf 0.4 --device 0 \
    --weights '/content/drive/MyDrive/PT/v9Animals.pt' \
    --source '/content/yolov9/saved_images'

    # Send the downloaded photo back to the user
    final_image_path = os.path.join("/content/yolov9/runs/detect/exp", image_filename)

    with open(final_image_path, 'rb') as photo_file:
        print(image_path)
        bot.send_photo(message.chat.id, photo_file)

    bot.reply_to(message, "Image saved and sent!")

@bot.message_handler(func=lambda message: True)
def handle_text_messages(message):
    bot.reply_to(message, "Please send an image.")

if __name__ == "__main__":
    bot.polling(none_stop=True)

Image Received
detect: weights=['/content/drive/MyDrive/PT/v9Animals.pt'], source=/content/yolov9/saved_images, data=data/coco128.yaml, imgsz=[1280, 1280], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=0, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 1e33dbb Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
gelan-c summary: 467 layers, 25418670 parameters, 0 gradients, 102.5 GFLOPs
image 1/1 /content/yolov9/saved_images/259957718_574.jpg: 1280x864 1 chicken, 100.4ms
Speed: 1.1ms pre-process, 100.4ms inference, 504.8ms NMS per image at shape (1, 3, 1280, 1280)
Results saved to runs/detect/exp
/content/yolov9/saved_images/259957718_574.jpg


In [ ]:
import gradio as gr
import os
import shutil
from PIL import Image
import subprocess
import logging

# Setup basic logging
logging.basicConfig(level=logging.INFO)

# Constants for paths and model
BASE_FOLDER = "/content/yolov9"
SAVED_IMAGES_FOLDER = os.path.join(BASE_FOLDER, "saved_images")
DETECT_FOLDER = os.path.join(BASE_FOLDER, "runs/detect")
MODEL_WEIGHTS = "/content/drive/MyDrive/PT/v9Animals.pt"

def clean_directories():
    """Remove all files and folders within the detect directory to start fresh each time."""
    shutil.rmtree(DETECT_FOLDER, ignore_errors=True)
    os.makedirs(DETECT_FOLDER, exist_ok=True)
    os.makedirs(SAVED_IMAGES_FOLDER, exist_ok=True)

def detect_objects(image):
    """Run object detection and return the image with detections and the text results."""
    clean_directories()
    image_path = os.path.join(SAVED_IMAGES_FOLDER, "input_image.jpg")
    output_image_path = os.path.join(DETECT_FOLDER, "exp", "input_image.jpg")

    # Save the input image
    image.save(image_path)

    # Run detection model
    subprocess.run([
        "python", "detect.py",
        "--img", "1280",
        "--conf", "0.4",
        "--device", "0",
        "--weights", MODEL_WEIGHTS,
        "--source", SAVED_IMAGES_FOLDER,
        "--project", DETECT_FOLDER,
        "--name", "exp",
        "--exist-ok",
        "--save-txt"  # Ensure this flag is included
    ], check=True)

    # Load the processed image
    result_image = Image.open(output_image_path) if os.path.exists(output_image_path) else image

    # Read detection results
    results_path = os.path.join(DETECT_FOLDER, "exp", "labels", "input_image.txt")
    detection_results = "No detections."
    if os.path.exists(results_path):
        with open(results_path, 'r') as file:
            detections = file.readlines()
        detection_results = "".join(detections)

    return result_image, detection_results

# Setup Gradio interface
iface = gr.Interface(
    fn=detect_objects,
    inputs=gr.Image(type="pil"),
    outputs=[gr.Image(type="pil"), gr.Textbox(label="Detection Results")],
    title="YOLOv9 Object Detection",
    description="Upload an image to detect objects using YOLOv9."
)

if __name__ == "__main__":
    iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://4cdfa4b73b100c46e9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
